## text (from pytesseract) to csv

In [ ]:
from dotenv import load_dotenv
import os


#### path
print("\n\n path ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ\n")
load_dotenv()
ROOT_DIR = os.getenv("ROOT_DIR")


pdf_image_path_01 = os.path.join(ROOT_DIR, "data", "sample_image_01.pdf")
pdf_image_path_05 = os.path.join(ROOT_DIR, "data", "sample_image_05.pdf")
pdf_image_path_20 = os.path.join(ROOT_DIR, "data", "sample_image_20.pdf")
pdf_image_path = os.path.join(ROOT_DIR, "data", "soil_suitability.PDF")

ocr_csv_path = os.path.join(ROOT_DIR, "data", "ocr_text.csv")


print("pwd :", os.getcwd())
print("input :", pdf_image_path)
print("output :", ocr_csv_path)

In [ ]:
import pytesseract
from pdf2image import convert_from_path
import json


#### env path
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
poppler_path = r"Z:\maeng_dir\sparta_camp\final_sparta\lib\poppler-24.08.0\Library\bin"


#### pdf to image
print("\n\n convert to images ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ\n")
images = convert_from_path(
                            pdf_image_path,
                            dpi = 300,
                            poppler_path=poppler_path,
                        )

In [ ]:
#### convert to dict
print("\n\n convert to text & dict ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ\n")
crop_dict = {}
for i, image in enumerate(images):
    # to str
    ocr_text = pytesseract.image_to_string(image, lang="kor+eng")
    
    #### crop name
    lines_list = ocr_text.split("\n")
    try:crop_name = lines_list[0].split(") ")[1]
    except:continue
    print("crop_name :", crop_name)
    
    #### parsing nut values
    for idx in range(1, 6):
        nut_val_str = lines_list[-idx]
        if "-" in nut_val_str and "     " in nut_val_str:
            val_list = nut_val_str.replace("|", "").replace(")", "").replace("(", "").split(" ")
            val_list = list(filter(None, val_list))
            crop_dict[crop_name] = val_list
            break
        
    try:print(i, "len vals :", len(crop_dict[crop_name]), "   val_list :", crop_dict[crop_name])
    except:print("fail parsing")
    
    

In [ ]:
import pandas as pd
import numpy as np

#### cloumns
col_list_09 = ["작물"] + "산도 전기전도도 질산태질소 유기물 유효인산 칼륨 칼슘 마그네슘 양이온교환용량".split(" ")
col_list_08 = ["작물"] + "산도 전기전도도 유기물 유효인산 칼륨 칼슘 마그네슘 양이온교환용량".split(" ")
col_list_07 = ["작물"] + "산도 유기물 유효인산 칼륨 칼슘 마그네슘 양이온교환용량".split(" ")

df = pd.DataFrame(columns=col_list_09, dtype='str')


#### convert dict to df
key_list = crop_dict.keys()
for crop_name in key_list:
    
    #### val_list
    val_list = crop_dict[crop_name]
    val_list = ",".join(val_list).replace("-", "~").split(",")
    
    len_vals = len(val_list)
    val_list = [[crop_name] + val_list]
    
    
    #### insert to df
    if len_vals == 7:
        df = pd.concat([df, pd.DataFrame(data = val_list, columns=col_list_07, dtype='str')], ignore_index=True)
    elif len_vals == 8:
        df = pd.concat([df, pd.DataFrame(data = val_list, columns=col_list_08, dtype='str')], ignore_index=True)
    elif len_vals == 9:
        df = pd.concat([df, pd.DataFrame(data = val_list, columns=col_list_09, dtype='str')], ignore_index=True)
    else:
        df = pd.concat([df, pd.DataFrame(data = [[crop_name]], columns=['작물'], dtype='str')], ignore_index=True)

#### save to csv
df.to_csv(ocr_csv_path, encoding="cp949", index=False)

In [ ]:
import pandas as pd
import os


#### read csv
ocr_whole_path = os.path.join(ROOT_DIR, "data", "ocr_whole.csv")
output_path = os.path.join(ROOT_DIR, "data", "ocr_tests.csv")

df = pd.read_csv(ocr_whole_path)
df.to_csv(ocr_whole_path, encoding="cp949", index=False)

In [ ]:
ocr_dict = {}

crop_list = """
                대파, 배추, 부추, 상추, 셀러리, 시금치, 신선초, 쑥갓, 양배추, 양상추,
                케일, 쪽파, 아스파라거스, 방울다다기, 근대, 치커리,
                감귤, 단감, 매실, 무화과, 배, 복숭아, 사과, 유자, 참다래, 포도, 대추,
                산딸기, 자두, 살구, 블랙베리, 샤인머스캣, 허니베리, 밤, 블루베리,
                뽕나무, 블랙커런트, 돌배나무, 플럼코트,
                가지, 고추, 딸기, 메론, 참외, 수박, 오이, 토마토, 피망, 호박, 단호박,
                맷돌호박, 애호박, 주키니 호박, 파프리카, 송이토마토, 수세미오이,
                여주, 울외, 애플수박,
                당근, 무, 생강, 우엉, 천마, 연근, 알타리무, 토란, 콜라비, 비트, 순무,
                겨자무, 야콘,
                논콩, 메밀, 밀, 보리, 수수, 옥수수, 조, 콩, 팥, 강낭콩, 기장, 녹두,
                동부콩, 쥐눈이콩, 귀리, 아마란스, 맥주보리, 완두, 율무, 작두콩,
                고사리, 곤달비, 곰취, 참취, 냉이, 달래, 씀바귀, 섬쑥부쟁이, 어수리,
                고려엉겅퀴, 고들빼기, 민들레, 눈개승마, 비름, 병풍쌈, 수리취,
                감자, 고구마,
                강활, 구기자, 도라지, 당귀, 마, 맥문동, 복분자, 산수유, 오미자, 인삼,
                황기, 더덕, 강황, 단삼, 땅두릅, 반하, 백지, 삼백초, 스테비아, 어성초,
                엉겅퀴, 일천궁, 지황, 황금, 황련, 삽주, 결명자, 잔대, 배암차즈기,
                더위지기, 감초, 곤약, 박하, 모링가, 차나무,
                들깨, 땅콩, 유채, 참깨,
                마늘, 양파,
                피, 수단그라스, 쥐보리, 청보리, 목초, 케나프,
                꽃도라지, 코스모스, 용담
            """
            
crop_list = crop_list.replace("\n", "").replace(" ", "").split(",")
crop_list


<토양적성등급 설정작물>

구분 작물명

경엽채류 대파, 배추, 부추, 상추, 셀러리, 시금치, 신선초, 쑥갓, 양배추, 양상추,

(16) 케일, 쪽파, 아스파라거스, 방울다다기, 근대, 치커리

감귤, 단감, 매실, 무화과, 배, 복숭아, 사과, 유자, 참다래, 포도, 대추,

과수

산딸기, 자두, 살구, 블랙베리, 샤인머스캣, 허니베리, 밤, 블루베리,

(23)

뽕나무, 블랙커런트, 돌배나무, 플럼코트

가지, 고추, 딸기, 메론, 참외, 수박, 오이, 토마토, 피망, 호박, 단호박,

과채류

맷돌호박, 애호박, 주키니 호박, 파프리카, 송이토마토, 수세미오이,

(20)

여주, 울외, 애플수박

근채류 당근, 무, 생강, 우엉, 천마, 연근, 알타리무, 토란, 콜라비, 비트, 순무,

(13) 겨자무, 야콘

곡류 논콩, 메밀, 밀, 보리, 수수, 옥수수, 조, 콩, 팥, 강낭콩, 기장, 녹두,

(20) 동부콩, 쥐눈이콩, 귀리, 아마란스, 맥주보리, 완두, 율무, 작두콩

산채류 고사리, 곤달비, 곰취, 참취, 냉이, 달래, 씀바귀, 섬쑥부쟁이, 어수리,

(16) 고려엉겅퀴, 고들빼기, 민들레, 눈개승마, 비름, 병풍쌈, 수리취

서류

감자, 고구마

(2)

강활, 구기자, 도라지, 당귀, 마, 맥문동, 복분자, 산수유, 오미자, 인삼,

약용작물 황기, 더덕, 강황, 단삼, 땅두릅, 반하, 백지, 삼백초, 스테비아, 어성초,

(35) 엉겅퀴, 일천궁, 지황, 황금, 황련, 삽주, 결명자, 잔대, 배암차즈기,

더위지기, 감초, 곤약, 박하, 모링가, 차나무

유지류

들깨, 땅콩, 유채, 참깨

(4)

인경채류

마늘, 양파

(2)

사료작물

피, 수단그라스, 쥐보리, 청보리, 목초, 케나프

(6)

화훼류

꽃도라지, 코스모스 용담

(3)

계: 160작물

* 감귤, 유채는 제주도 토양에 한하며, 논콩은 논에 한함

* 밑줄( ): 생리특성 유사작물 적용

-5-

